In [ ]:
from sec_edgar_downloader import Downloader
#needs to comply with sec privacy :(( )
dl = Downloader("test", "test@test.com")
import pandas as pd 
import time
import requests

#sec offers an api to get the daily or quartely indexes, metadata for all fillings
#there is no endpoint to specifically search by form type
#the actual data can only be downloaded with an accesion number and cik
#using cik fetched from daily index, fetch all fillings by company and select those with type D 
#download the actual file using url from the data line(the current data url returns the entire txt, we only need the actual xml file for the filing ) 
## Tom: added inputs
## Tom: Maybe add this lateremail = input("Enter your email for SEC requests (e.g., example@example.com): ")?
## Tom: It would be also nice to have the quarter variable derived from the other inputs. 
# //todo, save files to not fetch every time 
year = int(input("Enter the year in which you are interested (e.g., 2024). "))
quarter = int(input("Enter the quarter in which you are interested (e.g. 4). "))
month = int(input("Enter the month of interest in numerical format (e.g. 11)."))
day = int(input("Enter the day of interest in numerical format (e.g. 27)."))
date = f"{year}{month}{day}"
base_url = 'https://www.sec.gov/Archives/edgar/daily-index'
index_url = f'{base_url}/{year}/QTR{quarter}/company.{date}.idx'

base_full_index_url ='https://www.sec.gov/Archives/edgar/full-index'
full_index_url =  f'{base_full_index_url}/2024/QTR3/company.idx'

headers = {
    'User-Agent': 'Test (test@test.com)',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov',
    'Connection': 'keep-alive',
}
response = requests.get(index_url, headers=headers)
#response = requests.get(full_index_url, headers=headers)

if response.status_code == 403:
    print("Access denied. Ensure you have a proper User-Agent header.")
    exit()

response.raise_for_status() 


content = response.text

print(content)
lines = content.splitlines()


form_d_filings = []



header = ["Company Name", "Form Type", "CIK", "Date Filed", "File Name"]
#data lines skip 3 for daily index, 7 for full index 
#daily index returns idx file with all the fillings for the day, fixed width format, parse it into pandas 
records = [] 
#daily index formatting 

data_lines= lines[3:]
for line in data_lines:
    if line.strip():  
        company_name = line[:60].strip()
        form_type = line[60:71].strip()
        cik = line[71:82].strip()
        date_filed = line[82:92].strip()
        file_name = line[92:].strip()
        records.append([company_name, form_type, cik, date_filed, file_name])


#quarter index formatting 
'''
data_lines=lines[8:]
for line in data_lines:
    if line.strip():  # Ignore empty lines
        company_name = line[0:59].strip()
        form_type = line[59:72].strip()
        cik = line[72:87].strip()
        date_filed = line[87:102].strip()
        file_name = line[102:].strip()
        
        records.append([company_name, form_type, cik, date_filed, file_name])
        '''
print(records[1])
df = pd.DataFrame(records, columns=header)
print(response.text)
print(len(records))

form_d_df = df[df["Form Type"] == "D"] 
row_dict = form_d_df.iloc[1].to_dict()  
print("Row at index 1 as a dictionary:")
for key, value in row_dict.items():
    print(f"{key}: {value}")
#actual data can be downloaded with CIK and an accession number for the filling
form_d_df['Acession_number'] = form_d_df['File Name'].str.split('/').str[-1].str.replace('-', '').str.replace('.txt', '', regex=False)


base_url = "https://www.sec.gov/Archives/edgar/data/"
headers = {
    'User-Agent': 'Test (test@test.com)',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov',
    'Connection': 'keep-alive',
}

xml_data_list = []
for index, row in form_d_df.iterrows():
    print("working:" + row['CIK'])
    cik = row['CIK']
    accession_number = row['Acession_number']
    url = f"{base_url}{cik}/{accession_number}/primary_doc.xml"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            xml_data_list.append(response.text)
        else:
            xml_data_list.append(f"Failed for {url} with status code {response.status_code}")
    except Exception as e:
        xml_data_list.append(f"Error for {url}: {str(e)}")
print(len(form_d_df))

#//todo parse xml file to pandas 

In [ ]:
# Parsing the XML file to extract the relevant information

import xml.etree.ElementTree as ET
import plotly.express as px
import plotly.graph_objects as go
# Definine parsing function
def xml_parse(root):
    def extract_text(element, path):
        found = element.find(path)
        return found.text if found is not None else None

    # Parse primary issuer data
    primary_issuer = root.find(".//primaryIssuer")
    if primary_issuer is None:
        return None
    data_primary_issuer = {
        "Central Index Key": extract_text(primary_issuer, ".//cik"),
        "Name of the Entity": extract_text(primary_issuer, ".//entityName"),
        "Address": extract_text(primary_issuer, ".//street1"),
        "Address - Specification": extract_text(primary_issuer, ".//street2"),
        "City": extract_text(primary_issuer, ".//city"),
        "State or Country - Physical Location": extract_text(primary_issuer, ".//stateOrCountry"),
        "State or Country - Full - Physical Location": extract_text(primary_issuer, ".//stateOrCountryDescription"),
        "Zip Code / Postal Code": extract_text(primary_issuer, ".//zipCode"),
        "Issuer Phone Number": extract_text(primary_issuer, ".//issuerPhoneNumber"),
        "State or Country - Legal Jurisdiction": extract_text(primary_issuer, ".//jurisdictionOfInc"),
        "Previous Names Used by the Issuer": extract_text(primary_issuer, ".//issuerPreviousNameList/value"),
        "Previous Names from EDGAR": extract_text(primary_issuer, ".//edgarPreviousNameList/value"),
        "Type of the Entity": extract_text(primary_issuer, ".//entityType"),
        "Type of the Entity - Specification": extract_text(primary_issuer, ".//entityTypeOtherDesc"),
    }

    # Parse related persons data
    related_persons = root.findall(".//relatedPersonsList/relatedPersonInfo")
    data_related_persons = {}
    for i, related_person in enumerate(related_persons, start=1):
        data_related_persons[f"First Name - Person {i}"] = extract_text(related_person, ".//relatedPersonName/firstName"),
        data_related_persons[f"Last Name - Person {i}"] = extract_text(related_person, ".//relatedPersonName/lastName"),
        data_related_persons[f"Associated Address - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/street1"),
        data_related_persons[f"Associated Address - Specification - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/street2"),
        data_related_persons[f"Associated City - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/city"),
        data_related_persons[f"Associated State or Country - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/stateOrCountry"),
        data_related_persons[f"Associated State or Country - Full - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/stateOrCountryDescription"),
        data_related_persons[f"Associated Zip Code - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/zipCode"),

        relationships = related_person.findall(".//relatedPersonRelationshipList/relationship")
        for j, relationship in enumerate(relationships, start=1):
            data_related_persons[f"Relationship {j} with the Entity - Person {i}"] = relationship.text if relationship is not None else None

        data_related_persons[f"Clarification of the relationship - Person {i}"] = extract_text(related_person, ".//relationshipClarification"),

    # Parse offering data
    offering_data = root.find(".//offeringData")
    data_offering_data = {
        "Industry": extract_text(offering_data, ".//industryGroup/industryGroupType"),
        "Revenue Range of the Entity": extract_text(offering_data, ".//issuerSize/revenueRange"),
        "Federal Exemption or Exclusions Claimed by the Entity": ", ".join([item.text for item in offering_data.findall(".//federalExemptionsExclusions/item") if item is not None]) if offering_data is not None else None,
        "Is the Entry an Amendment to Another Filing?": extract_text(offering_data, ".//typeOfFiling/newOrAmendment/isAmendment"),
        "Date of the First Sale Under this Filing": extract_text(offering_data, ".//typeOfFiling/dateOfFirstSale/value"),
        "Is the Offering Duration Intended to be More Than a Year?": extract_text(offering_data, ".//durationOfOffering/moreThanOneYear"),
        "Is the Entity Offering Equity": extract_text(offering_data, ".//typesOfSecuritiesOffered/isEquityType"),
        "Is the Entity Creating or Managing a Pooled Fund": extract_text(offering_data, ".//typesOfSecuritiesOffered/isPooledInvestmentFundType"),
        "Is the Offering Part of a Business Combination Transaction?": extract_text(offering_data, ".//isPooledInvestmentFundType/isBusinessCombinationTransaction"),
        "Is the Offering Part of a Business Combination Transaction? - Clarification": extract_text(offering_data, ".//businessCombinationTransaction/clarificationOfResponse"),
        "Minimum Investment Amount Accepted": extract_text(offering_data, ".//minimumInvestmentAccepted"),
        #"Entities Compensated for Selling the Securities": extract_text(offering_data, ".//salesCompensationList"),
        # The line above is commented out because we don't see much use in it
        "Total Amount Offered": extract_text(offering_data, ".//offeringSalesAmounts/totalOfferingAmount"),
        "Total Amount Sold So Far": extract_text(offering_data, ".//offeringSalesAmounts/totalAmountSold"),
        "Total Amount Remaining to be sold": extract_text(offering_data, ".//offeringSalesAmounts/totalRemaining"),
        "Clarifications Regarding the Offering and Sales amounts": extract_text(offering_data, ".//offeringSalesAmounts/clarificationOfResponse"),
        "Are Non-accredited Investors Participating in the Offering?": extract_text(offering_data, ".//investors/hasNonAccreditedInvestors"),
        "Number of Investors Who Already Invested Under Current Offering": extract_text(offering_data, ".//investors/totalNumberAlreadyInvested"),
        "Commision Amount Paid for Sales": extract_text(offering_data, ".//salesCommissionsFindersFees/salesCommissions/dollarAmount"),
        "Finder's Fees Amount Paid": extract_text(offering_data, ".//salesCommissionsFindersFees/findersFees/dollarAmount"),
        "Clarification Regarding the Finder's Fees Amount Paid": extract_text(offering_data, ".//salesCommissionsFindersFees/clarificationOfResponse"),
        "Amount of the Raised Capital Already Used for a Purpose": extract_text(offering_data, ".//useOfProceeds/grossProceedsUsed/dollarAmount"),
        "Clarification Regarding the Amount of the Raised Capital Already Used for a Purpose": extract_text(offering_data, ".//useOfProceeds/clarificationOfResponse"),
        "Name of the Entity Representative Signing the Filing": extract_text(offering_data, ".//signatureBlock/authorizedRepresentative"),
        "Name of the Issuer Entity": extract_text(offering_data, ".//signatureBlock/signature/issuerName"),
        "Name of the Person Signing": extract_text(offering_data, ".//signatureBlock/signature/signatureName"),
        "Name of the Person Signing - Affirmation": extract_text(offering_data, ".//signatureBlock/signature/nameOfSigner"),
        "Title of the Person Signing": extract_text(offering_data, ".//signatureBlock/signature/signatureTitle"),
        "Date the Form Was Signed": extract_text(offering_data, ".//signatureBlock/signature/signatureDate")
    }
    # Combine all data
    data = {**data_primary_issuer, **data_related_persons, **data_offering_data}
    return data

data_fin = []

for information in xml_data_list:
    print(information)
    root = ET.fromstring(information)
    data_fin.append(xml_parse(root))

df_main = pd.DataFrame(data_fin)
#pd.set_option('display.max_columns', 500)
# A very inefficient way to reorganize the columns
#df_main.reindex((["Central Index Key", "Name of the Entity", "Address", "Address - Specification", "City", "State or Country - Physical Location", "State or Country - Full - Physical Location", "Zip Code / Postal Code", "Issuer Phone Number", "State or Country - Legal Jurisdiction", "Previous Names Used by the Issuer", "Previous Names from EDGAR", "Type of the Entity", "Type of the Entity - Specification", "Industry", "Revenue Range of the Entity", "Federal Exemption or Exclusions Claimed by the Entity", "Is the Entry an Amendment to Another Filing?", "Date of the First Sale Under this Filing", "Is the Offering Duration Intended to be More Than a Year?", "Is the Entity Offering Equity", "Is the Entity Creating or Managing a Pooled Fund", "Is the Offering Part of a Business Combination Transaction?", "Is the Offering Part of a Business Combination Transaction? - Clarification", "Minimum Investment Amount Accepted", "Entities Compensated for Selling the Securities", "Total Amount Offered", "Total Amount Sold So Far", "Total Amount Remaining to be sold", "Clarifications Regarding the Offering and Sales amounts", "Are Non-accredited Investors Participating in the Offering?", "Number of Investors Who Already Invested Under Current Offering", "Commision Amount Paid for Sales", "Finder's Fees Amount Paid", "Clarification Regarding the Finder's Fees Amount Paid", "Amount of the Raised Capital Already Used for a Purpose", "Clarification Regarding the Amount of the Raised Capital Already Used for a Purpose", "Name of the Entity Representative Signing the Filing", "Name of the Issuer Entity", "Name of the Person Signing", "Name of the Person Signing - Affirmation", "Title of the Person Signing", "Date the Form Was Signed", "First Name - Person 1", "Last Name - Person 1", "Associated Address - Person 1", "Associated Address - Specification - Person 1", "Associated City - Person 1", "Associated State or Country - Person 1", "Associated State or Country - Full - Person 1", "Associated Zip Code - Person 1", "Relationship 1 with the Entity - Person 1", "Relationship 2 with the Entity - Person 1", "Relationship 3 with the Entity - Person 1", "Clarification of the relationship - Person 1", "First Name - Person 2", "Last Name - Person 2", "Associated Address - Person 2", "Associated Address - Specification - Person 2", "Associated City - Person 2", "Associated State or Country - Person 2", "Associated State or Country - Full - Person 2", "Associated Zip Code - Person 2", "Relationship 1 with the Entity - Person 2", "Relationship 2 with the Entity - Person 2", "Clarification of the relationship - Person 2", "First Name - Person 3", "Last Name - Person 3", "Associated Address - Person 3", "Associated Address - Specification - Person 3", "Associated City - Person 3", "Associated State or Country - Person 3", "Associated State or Country - Full - Person 3", "Associated Zip Code - Person 3", "Relationship 1 with the Entity - Person 3", "Relationship 2 with the Entity - Person 3", "Clarification of the relationship - Person 3", "First Name - Person 4", "Last Name - Person 4", "Associated Address - Person 4", "Associated Address - Specification - Person 4", "Associated City - Person 4", "Associated State or Country - Person 4", "Associated State or Country - Full - Person 4", "Associated Zip Code - Person 4", "Relationship 1 with the Entity - Person 4", "Clarification of the relationship - Person 4", "First Name - Person 5", "Last Name - Person 5", "Associated Address - Person 5", "Associated Address - Specification - Person 5", "Associated City - Person 5", "Associated State or Country - Person 5", "Associated State or Country - Full - Person 5", "Associated Zip Code - Person 5", "Relationship 1 with the Entity - Person 5", "Clarification of the relationship - Person 5"]), axis=1)

# Cleaning the data
df_main = df_main.fillna("N/A")
for column in df_main.columns:
    df_main[column] = df_main[column].apply(lambda x: x[0] if isinstance(x, tuple) else x)

df_main = df_main.replace("None", "N/A")

redundant_strings = ["/s/", "/bem/", "c/o", "n/a"]
for x in redundant_strings:
    df_main = df_main.replace(x, "", regex=True)

df_main = df_main.replace("", "N/A")
df_main = df_main.fillna("N/A")





In [19]:
import zipfile
import pandas as pd
import plotly.express as px
import os
#Data filter for growth in fillings by US state, input the state abbreviation for the filter to work 

years = list(range(2014, 2025))  # From 2014 to 2024
quarters = ["QTR1", "QTR2", "QTR3", "QTR4"]

# State abbreviations and full names
all_states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
state_full_names = [
    "ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO", "CONNECTICUT", 
    "DELAWARE", "FLORIDA", "GEORGIA", "HAWAII", "IDAHO", "ILLINOIS", "INDIANA", "IOWA", 
    "KANSAS", "KENTUCKY", "LOUISIANA", "MAINE", "MARYLAND", "MASSACHUSETTS", "MICHIGAN", 
    "MINNESOTA", "MISSISSIPPI", "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA", "NEW HAMPSHIRE", 
    "NEW JERSEY", "NEW MEXICO", "NEW YORK", "NORTH CAROLINA", "NORTH DAKOTA", "OHIO", 
    "OKLAHOMA", "OREGON", "PENNSYLVANIA", "RHODE ISLAND", "SOUTH CAROLINA", "SOUTH DAKOTA", 
    "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON", "WEST VIRGINIA", 
    "WISCONSIN", "WYOMING"
]

state_name_to_abbreviation = dict(zip(state_full_names, all_states))

zip_file_path = "Data.zip"
extract_path = "extracted_data"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

all_data = []

for year in years:
    for quarter in quarters:
        csv_file_name = f"{year}_{quarter}.csv"
        file_path = os.path.join(extract_path, csv_file_name)
        
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)

            jurisdiction_counts = (
                df["State or Country - Legal Jurisdiction"]
                .map(state_name_to_abbreviation)
                .value_counts()
                .reset_index()
            )
            jurisdiction_counts.columns = ["State", "JurisdictionCount"]

            state_counts_simple = (
                df["State or Country - Physical Location"]
                .value_counts()
                .reset_index()
            )
            state_counts_simple.columns = ["State", "TotalCount"]
            state_counts_simple["TotalCount"] = state_counts_simple["TotalCount"].fillna(0).astype(int)

            merged_counts = pd.merge(
                jurisdiction_counts, state_counts_simple, on="State", how="outer"
            ).fillna(0)

            merged_counts["Percentage"] = (
                (merged_counts["JurisdictionCount"] / merged_counts["TotalCount"]) * 100
            ).fillna(0)

            merged_counts["Year"] = year
            merged_counts["Quarter"] = quarter
            
            all_data.append(merged_counts)

final_df = final_df.sort_values(by=["State", "Year"])

final_df["PercentageGrowth"] = (
    (final_df["Percentage"] - final_df.groupby("State")["Percentage"].shift(4)) /
    final_df.groupby("State")["Percentage"].shift(4)
) * 100

final_df["PercentageGrowth"] = final_df["PercentageGrowth"].fillna(0)
selected_state = input("Enter the state abbreviation (e.g., CA, TX, NY): ").upper()

state_data = final_df[final_df["State"] == selected_state]

if state_data.empty:
    print(f"No data found for state: {selected_state}")
else:
    
    fig_growth = px.line(
        state_data,
        x="Year",
        y="PercentageGrowth",
        title=f"Yearly Jurisdiction Percentage Growth in {selected_state} (2014-2024)",
        labels={"PercentageGrowth": "Percentage Growth (%)"},
    )


fig_growth.show()


/var/folders/d2/m57xcgf11tb1brc3mk8jsgjm0000gn/T/ipykernel_91254/861254386.py:47: DtypeWarning:

Columns (80,156,166,176,178,186,188,196,198,206,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,323,324,325,326,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426

In [1]:
import zipfile
import pandas as pd
import plotly.express as px
import os
all_states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
state_full_names = [
    "ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO", "CONNECTICUT", 
    "DELAWARE", "FLORIDA", "GEORGIA", "HAWAII", "IDAHO", "ILLINOIS", "INDIANA", "IOWA", 
    "KANSAS", "KENTUCKY", "LOUISIANA", "MAINE", "MARYLAND", "MASSACHUSETTS", "MICHIGAN", 
    "MINNESOTA", "MISSISSIPPI", "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA", "NEW HAMPSHIRE", 
    "NEW JERSEY", "NEW MEXICO", "NEW YORK", "NORTH CAROLINA", "NORTH DAKOTA", "OHIO", 
    "OKLAHOMA", "OREGON", "PENNSYLVANIA", "RHODE ISLAND", "SOUTH CAROLINA", "SOUTH DAKOTA", 
    "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON", "WEST VIRGINIA", 
    "WISCONSIN", "WYOMING"
]

# Extract the CSV file from the ZIP archive
zip_file_path = "Data.zip"
csv_file_name = "2022_QTR4.csv"
extract_path = "extracted_data" 
zip_file_path = "Data.zip"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

print("Files inside ZIP:", file_list)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extract(csv_file_name, "extracted_data")

# Read the extracted CSV file
df = pd.read_csv(f"extracted_data/{csv_file_name}")

state_name_to_abbreviation = dict(zip(state_full_names, all_states))

jurisdiction_counts = (
    df["State or Country - Legal Jurisdiction"]
    .map(state_name_to_abbreviation)
    .value_counts()  
    .reset_index()
)
jurisdiction_counts.columns = ["State", "JurisdictionCount"]
state_counts_simple = (
    df["State or Country - Physical Location"]
    .value_counts()
    .reset_index()
)
state_counts_simple.columns = ["State", "TotalCount"]
state_counts_simple["TotalCount"] = state_counts_simple["TotalCount"].fillna(0).astype(int)
print(state_counts_simple)

merged_counts = pd.merge(
    jurisdiction_counts, state_counts_simple, on="State", how="outer"
).fillna(0)

merged_counts["Percentage"] = (
    (merged_counts["JurisdictionCount"] / merged_counts["TotalCount"]) * 100
).fillna(0)

# Ensure all states are represented in the DataFrame with default count 0
#state_counts_simple = df["State or Country - Physical Location"].value_counts().reset_index()
#state_counts_simple.columns = ["State", "Count"]
#state_counts_simple["Count"] = state_counts_simple["Count"].fillna(0).astype(int)

merged_counts = merged_counts[["State", "Percentage"]]
print(merged_counts.head(55))

fig_simple = px.choropleth(
    merged_counts,
    locations="State",
    locationmode="USA-states",
    color="Percentage",
    range_color=(0,100),
    scope="usa",
    color_continuous_scale="Viridis",
    title="Count of D type fillings by US state, state of filling vs company location"
)

# Displaying the map
fig_simple.show()

Files inside ZIP: ['2014_QTR1.csv', '2014_QTR2.csv', '2014_QTR3.csv', '2014_QTR4.csv', '2015_QTR1.csv', '2015_QTR2.csv', '2015_QTR3.csv', '2015_QTR4.csv', '2016_QTR1.csv', '2016_QTR2.csv', '2016_QTR3.csv', '2016_QTR4.csv', '2017_QTR1.csv', '2017_QTR2.csv', '2017_QTR3.csv', '2017_QTR4.csv', '2018_QTR1.csv', '__MACOSX/._2018_QTR1.csv', '2018_QTR2.csv', '2018_QTR3.csv', '2018_QTR4.csv', '2019_QTR1.csv', '2019_QTR2.csv', '2019_QTR3.csv', '2019_QTR4.csv', '2020_QTR1.csv', '2020_QTR2.csv', '2020_QTR3.csv', '2020_QTR4.csv', '2021_QTR1.csv', '2021_QTR2.csv', '2021_QTR3.csv', '2021_QTR4.csv', '2022_QTR1.csv', '2022_QTR2.csv', '2022_QTR3.csv', '2022_QTR4.csv', '2023_QTR1.csv', '2023_QTR2.csv', '2023_QTR3.csv', '2023_QTR4.csv', '2024_QTR1.csv', '2024_QTR2.csv', '2024_QTR3.csv', '2024_QTR4.csv', '__MACOSX/._2024_QTR4.csv']


/var/folders/fv/pg6m92317192zjv46jbqrk_80000gn/T/ipykernel_8613/1645582559.py:38: DtypeWarning: Columns (67,154,164,169,180,182,190,192,200,202,206,210,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,

    State  TotalCount
0      WA        1251
1      CA        1200
2      NY         991
3      TX         769
4      FL         446
..    ...         ...
102    J5           1
103    R1           1
104    J3           1
105    1M           1
106    R5           1

[107 rows x 2 columns]
   State   Percentage
0     1M     0.000000
1     2M     0.000000
2     A0     0.000000
3     A1     0.000000
4     A3     0.000000
5     A5     0.000000
6     A6     0.000000
7     A8     0.000000
8     A9     0.000000
9     AK   114.285714
10    AL    33.333333
11    AR    78.571429
12    AZ    38.383838
13    C0     0.000000
14    C1     0.000000
15    C3     0.000000
16    C4     0.000000
17    C7     0.000000
18    C8     0.000000
19    CA     9.833333
20    CO    34.482759
21    CT     5.785124
22    D0     0.000000
23    D5     0.000000
24    D8     0.000000
25    DC     0.000000
26    DE  2095.000000
27    E9     0.000000
28    F3     0.000000
29    F4     0.000000
30    F8     0.000000
31    FL

In [2]:
import pandas as pd 
import zipfile
import  plotly.express as px
import plotly.graph_objects as go
zip_file_path = "Data.zip"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("extracted_data") 


csv_file_path = "extracted_data/2022_QTR3.csv"
df = pd.read_csv(csv_file_path)
total_income_by_state = (
    df.groupby(df["State or Country - Physical Location"])["Total Amount Sold So Far"]
    .sum()
    .reset_index()
)

total_income_by_state.columns = ["State", "TotalIncome" ]

# Create the choropleth map
fig_income = px.choropleth(
    total_income_by_state,
    locations="State",
    locationmode="USA-states",
    color="TotalIncome",
    scope="usa",
    color_continuous_scale="Viridis",
    title="Total Income Raised by State",
    labels={"TotalIncome": "Total Income ($)"}
)

fig_income.show()

/var/folders/fv/pg6m92317192zjv46jbqrk_80000gn/T/ipykernel_8613/3541256687.py:11: DtypeWarning:

Columns (123,125,133,135,143,145,149,157,159,161,162,163,171,173,181,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,323,324,325,326,327,328,329,330,331,333,334,335,336,337,338,339,340,341,343,344,345,346,347,348,349,350,351,353,354,355,356,357,358,359,360,361,362,363,364,365,367,368,369,370,371,372,373,374,375,377,378,379,380,381,382,383,384,385,387,388,389,390,391,392,393,394,395,397,398,39

In [4]:
def versatile_filter(df):
    global filtered_df
    # Create an "all True" Series that we can use as our starting mask
    mask_state = pd.Series(True, index=df.index)
    mask_subed = pd.Series(True, index=df.index)
    mask_amount = pd.Series(True, index=df.index)
    mask_equity = pd.Series(True, index=df.index)
    mask_industry = pd.Series(True, index=df.index)
    
    applied_filters = []

    # Ask the user whether they want to filter the data at all
    while True:
        user_filter = input("Would you like to filter the data? (yes/no): ").strip().lower()
        if user_filter in ["yes", "no"]:
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")
    if user_filter != "yes":
        print("Displaying sample of the full data:")
        filtered_df = df
        return filtered_df.sample(n=min(10, len(df)))[["Central Index Key", "Name of the Entity", "Address", "Address - Specification", "City", "State or Country - Physical Location", "State or Country - Full - Physical Location", "Zip Code / Postal Code", "Issuer Phone Number", "State or Country - Legal Jurisdiction", "Minimum Investment Amount Accepted", "Total Amount Offered", "Total Amount Sold So Far", "Total Amount Remaining to be sold", "Clarifications Regarding the Offering and Sales amounts", "Is the Entity Offering Equity", "Industry"]]

    # List of available filters
    available_filters = ["state", "fully_subscribed", "amount", "equity", "industry"]

    # Loop: let the user update or add filters until they are satisfied
    while True:
        print("\nAvailable filters:")
        print("  state             : Filter by state abbreviation")
        print("  fully_subscribed  : Filter by fully subscribed offerings")
        print("  amount            : Filter by offered funding amount range")
        print("  equity            : Filter by equity offerings")
        print("  industry          : Filter by industry")
        chosen_filter = input("Which filter would you like to apply/update? (state/fully_subscribed/amount/equity/industry): ").strip().lower()
        
        if chosen_filter not in available_filters:
            print("Invalid filter type. Please choose one of the available filters.")
            continue

        # Filter by state
        if chosen_filter == "state":
            action = input("Would you like to apply or delete the state filter? (apply/delete): ").strip().lower()
            while action not in ["apply", "delete"]:
                action = input("Invalid input. Please enter 'apply' or 'delete': ").strip().lower()
            if action == "delete":
                mask_state = pd.Series(True, index=df.index)
                applied_filters = [f for f in applied_filters if not f.startswith("State:")]
                print("State filter deleted.")
            else:
                state_abbreviations = input("Enter one or more state abbreviations separated by commas (e.g., CA,TX,NY): ").upper().split(',')
                # Ensure that the abbreviations are valid (assuming all_states is defined)
                state_abbreviations = [abbr.strip() for abbr in state_abbreviations if abbr.strip() in all_states]
                while not state_abbreviations:
                    state_abbreviations = input("Invalid state abbreviations. Please try again (e.g., CA,TX,NY): ").upper().split(',')
                    state_abbreviations = [abbr.strip() for abbr in state_abbreviations if abbr.strip() in all_states]
                # Update the state mask
                mask_state = df["State or Country - Physical Location"].isin(state_abbreviations)
                applied_filters = [f for f in applied_filters if not f.startswith("State:")]
                applied_filters.append(f"State: {', '.join(state_abbreviations)}")
                print(f"State filter set to: {', '.join(state_abbreviations)}")
        
        # Filter by fully subscribed offerings
        elif chosen_filter == "fully_subscribed":
            action = input("Would you like to apply or delete the fully subscribed filter? (apply/delete): ").strip().lower()
            while action not in ["apply", "delete"]:
                action = input("Invalid input. Please enter 'apply' or 'delete': ").strip().lower()
            if action == "delete":
                mask_subed = pd.Series(True, index=df.index)
                applied_filters = [f for f in applied_filters if not f.startswith("Fully Subscribed:")]
                print("Fully subscribed filter deleted.")
            else:
                subed = input("Are you interested only in fully subscribed offerings? (yes/no): ").strip().lower()
                while subed not in ["yes", "no"]:
                    subed = input("Invalid input. Please enter 'yes' or 'no': ").strip().lower()
                # Note: adjust the condition below based on how your data represents amounts.
                if subed == "yes":
                    mask_subed = (df["Total Amount Remaining to be sold"] == 0)
                    applied_filters = [f for f in applied_filters if not f.startswith("Fully Subscribed:")]
                    applied_filters.append("Fully Subscribed: Yes")
                    print("Filtering for fully subscribed offerings only.")
                else:
                    mask_subed = (df["Total Amount Remaining to be sold"] != 0)
                    applied_filters = [f for f in applied_filters if not f.startswith("Fully Subscribed:")]
                    applied_filters.append("Fully Subscribed: No")
                    print("Filtering for offerings that are not fully subscribed.")
    
        # Filter by funding amount range
        elif chosen_filter == "amount":
            action = input("Would you like to apply or delete the amount filter? (apply/delete): ").strip().lower()
            while action not in ["apply", "delete"]:
                action = input("Invalid input. Please enter 'apply' or 'delete': ").strip().lower()
            if action == "delete":
                mask_amount = pd.Series(True, index=df.index)
                applied_filters = [f for f in applied_filters if not f.startswith("Amount:")]
                print("Amount filter deleted.")
            else:
                while True:
                    try:
                        min_val = int(input("Enter the minimum funding amount in USD: "))
                        break
                    except ValueError:
                        print("Invalid input. Please enter an integer value for the minimum funding amount.")
                while True:
                    max_input = input("Enter the maximum funding amount in USD (or type 'infinite' for no upper limit): ").strip().lower()
                    if max_input == "infinite":
                        max_val = float('inf')
                        break
                    try:
                        max_val = int(max_input)
                        break
                    except ValueError:
                        print("Invalid input. Please enter an integer value for the maximum funding amount or 'infinite'.")
                if min_val > max_val:
                    print("The minimum funding amount cannot be greater than the maximum. Amount filter not applied.")
                else:
                    mask_amount = (df["Total Amount Offered"] >= min_val) & (df["Total Amount Offered"] <= max_val)
                    applied_filters = [f for f in applied_filters if not f.startswith("Amount:")]
                    applied_filters.append(f"Amount: {min_val} to {max_val}")
                    print(f"Filtering for funding amounts between {min_val} and {max_val} USD.")

        # Filter by equity offerings
        elif chosen_filter == "equity":
            action = input("Would you like to apply or delete the equity filter? (apply/delete): ").strip().lower()
            while action not in ["apply", "delete"]:
                action = input("Invalid input. Please enter 'apply' or 'delete': ").strip().lower()
            if action == "delete":
                mask_equity = pd.Series(True, index=df.index)
                applied_filters = [f for f in applied_filters if not f.startswith("Equity:")]
                print("Equity filter deleted.")
            else:
                equity = input("Are you interested only in offerings of equity or offerings excluding equity? (only_equity/excluding_equity): ").strip().lower()
                while equity not in ["only_equity", "excluding_equity"]:
                    equity = input("Invalid input. Please enter 'only_equity' or 'excluding_equity': ").strip().lower()
                if equity == "only_equity":
                    mask_equity = df["Is the Entity Offering Equity"] == True
                    applied_filters = [f for f in applied_filters if not f.startswith("Equity:")]
                    applied_filters.append("Equity: Only Equity")
                    print("Filtering for offerings of equity only.")
                else:
                    mask_equity = df["Is the Entity Offering Equity"] != True
                    applied_filters = [f for f in applied_filters if not f.startswith("Equity:")]
                    applied_filters.append("Equity: Excluding Equity")
                    print("Filtering for offerings excluding equity.")
        
        # Filter by industry
        elif chosen_filter == "industry":
            action = input("Would you like to apply or delete the industry filter? (apply/delete): ").strip().lower()
            while action not in ["apply", "delete"]:
                action = input("Invalid input. Please enter 'apply' or 'delete': ").strip().lower()
            if action == "delete":
                mask_industry = pd.Series(True, index=df.index)
                applied_filters = [f for f in applied_filters if not f.startswith("Industry:")]
                print("Industry filter deleted.")
            else:
                while True:
                    industry_input = input("Enter one or more industries separated by commas (or type 'list' to see all industries) - be case sensitive: ").strip()
                    if industry_input.lower() == "list":
                        print("Available industries:")
                        for industry in industries:
                            print(f"  {industry}")
                        continue
                    industry_list = [ind.strip() for ind in industry_input.split(',') if ind.strip() in industries]
                    if not industry_list:
                        print("Invalid industries. Please try again.")
                    else:
                        break
                mask_industry = df["Industry"].isin(industry_list)
                applied_filters = [f for f in applied_filters if not f.startswith("Industry:")]
                applied_filters.append(f"Industry: {', '.join(industry_list)}")
                print(f"Industry filter set to: {', '.join(industry_list)}")

        # Combine all the masks
        overall_mask = mask_state & mask_subed & mask_amount & mask_equity & mask_industry

        # Show a sample of the filtered data
        filtered_df = df[overall_mask]
        if filtered_df.empty:
            print("\nNo data matches the current filter criteria.")
        else:
            print("\nHere is a sample of the filtered data:")
            print("Applied Filters- " + "; ".join(applied_filters))
            display(filtered_df.sample(n=min(10, len(filtered_df)))[["Central Index Key", "Name of the Entity", "Address", "Address - Specification", "City", "State or Country - Physical Location", "State or Country - Full - Physical Location", "Zip Code / Postal Code", "Issuer Phone Number", "State or Country - Legal Jurisdiction", "Minimum Investment Amount Accepted", "Total Amount Offered", "Total Amount Sold So Far", "Total Amount Remaining to be sold", "Clarifications Regarding the Offering and Sales amounts", "Is the Entity Offering Equity", "Industry"]])
        
        # Ask if the user wants to apply/update another filter
        continue_filter = input("Would you like to apply/update another filter? (yes/no): ").strip().lower()
        while continue_filter not in ["yes", "no"]:
                continue_filter = input("Invalid input. Please enter 'yes' or 'no': ").strip().lower()
        if continue_filter != "yes":
            break

    print("\nDisplaying sample of the final filtered data:")
    return filtered_df.sample(n=min(10, len(filtered_df)))

versatile_filter(df)



Available filters:
  state             : Filter by state abbreviation
  fully_subscribed  : Filter by fully subscribed offerings
  amount            : Filter by offered funding amount range
  equity            : Filter by equity offerings
  industry          : Filter by industry
State filter set to: CA, TX

Here is a sample of the filtered data:
Applied Filters- State: CA, TX


,Central Index Key,Name of the Entity,Address,Address - Specification,City,State or Country - Physical Location,State or Country - Full - Physical Location,Zip Code / Postal Code,Issuer Phone Number,State or Country - Legal Jurisdiction,Minimum Investment Amount Accepted,Total Amount Offered,Total Amount Sold So Far,Total Amount Remaining to be sold,Clarifications Regarding the Offering and Sales amounts,Is the Entity Offering Equity,Industry
3144,1937931,ExchangeRight Net-Leased Portfolio 57 DST,"1055 E. COLORADO BLVD., SUITE 310",NaN,PASADENA,CA,CALIFORNIA,91106,855-317-4448,DELAWARE,100000,30450000,0,30450000,NaN,True,Commercial
6832,1936761,"Pravana Capital Partners Fund I, LP",824 EUCLID STREET,NaN,SAN FRANCISCO,CA,CALIFORNIA,94118,714-926-3199,CALIFORNIA,0,25000000,3000000,22000000,NaN,True,Investing
7200,1935038,Ramani Corp,311 WISTERIA DR,NaN,EAST PALO ALTO,CA,CALIFORNIA,94303,720-238-2873,DELAWARE,0,13778049,10097917,3680132,NaN,True,Other Technology
291,1909675,"AH Games Fund I-B, L.P.","2865 SAND HILL ROAD, SUITE 101",NaN,MENLO PARK,CA,CALIFORNIA,94025,(650) 798-3900,DELAWARE,0,Indefinite,0,Indefinite,NaN,NaN,Pooled Investment Fund
8056,1918088,SparkLabs Pakistan Fund I LLC,3790 EL CAMINO REAL,UNIT #570,PALO ALTO,CA,CALIFORNIA,94306,6504545244,DELAWARE,25000,700000,700000,0,NaN,NaN,Pooled Investment Fund
4401,1938639,"IREG Firewheel III Investors, LLC","3110 W. SOUTHLAKE BLVD., SUITE 120",NaN,SOUTHLAKE,TX,TEXAS,76092,(817) 837-0564,TEXAS,0,2156946,2156946,0,NaN,True,Other Real Estate
8522,1924612,TechTown Industrial L1 LLC,11408 HOLLISTER DRIVE,NaN,AUSTIN,TX,TEXAS,78739,512-484-8152,TEXAS,25000,3000000,1725000,1275000,NaN,True,Commercial
2352,1942206,Crown Titan #2 Joint Venture,4024 NAZARENE DRIVE,NaN,CARROLLTON,TX,TEXAS,75010,972-395-1133,TEXAS,18375,7350000,0,7350000,NaN,NaN,Oil and Gas
3913,1944064,"GrayMatter Robotics, Inc.",349 W. 168TH STREET,NaN,GARDENA,CA,CALIFORNIA,90248,(818)-794-0874,DELAWARE,1,20034614,20034614,0,NaN,True,Other Technology
8142,1944968,"Stellifi Fund I, LP","C/O STELLIFI I GP, LLC","2630 EXPOSITION BOULEVARD, SUITE 214",AUSTIN,TX,TEXAS,78703,512-871-0561,DELAWARE,0,35000000,0,35000000,NaN,NaN,Pooled Investment Fund



Available filters:
  state             : Filter by state abbreviation
  fully_subscribed  : Filter by fully subscribed offerings
  amount            : Filter by offered funding amount range
  equity            : Filter by equity offerings
  industry          : Filter by industry
Filtering for offerings of equity only.

Here is a sample of the filtered data:
Applied Filters- State: CA, TX; Equity: Only Equity


,Central Index Key,Name of the Entity,Address,Address - Specification,City,State or Country - Physical Location,State or Country - Full - Physical Location,Zip Code / Postal Code,Issuer Phone Number,State or Country - Legal Jurisdiction,Minimum Investment Amount Accepted,Total Amount Offered,Total Amount Sold So Far,Total Amount Remaining to be sold,Clarifications Regarding the Offering and Sales amounts,Is the Entity Offering Equity,Industry
4308,1938788,"ICONIQ Strategic Partners VII Co-Invest FO-B, ...",394 PACIFIC AVENUE,SECOND FLOOR,SAN FRANCISCO,CA,CALIFORNIA,94111,415-967-7763,DELAWARE,0,Indefinite,0,Indefinite,NaN,True,Pooled Investment Fund
2901,1767654,"Elite Beverage International, Inc.",970 NORTH TUSTIN AVENUE,SUITE 100,ANAHEIM,CA,CALIFORNIA,92807,6574560082,CALIFORNIA,100000,20000000,0,20000000,NaN,True,Other
9420,1943148,Zerion Inc,"50 CALIFORNIA STREET, SUITE 1500",NaN,SAN FRANCISCO,CA,CALIFORNIA,94111,657-214-1592,DELAWARE,0,14999848,10494974,4504874,NaN,True,Other Technology
8,1942481,10101 Forum Investors LLC,"300 CRESCENT COURT, SUITE 1800",NaN,DALLAS,TX,TEXAS,75201,(630) 333-8428,TEXAS,25000,2900000,2900000,0,NaN,True,Residential
889,1938924,Art is Love SPV I LLC,2519 FAIRMOUNT STREET,NaN,DALLAS,TX,TEXAS,75201,(469) 351-3471,DELAWARE,0,19133560,19133560,0,NaN,True,Pooled Investment Fund
6686,1948952,"PeakMetrics, Inc.",777 S. ALAMEDA,FLOOR 2,LOS ANGELES,CA,CALIFORNIA,90021,323-553-5888,DELAWARE,0,Indefinite,1889811,Indefinite,NaN,True,Other Technology
7200,1935038,Ramani Corp,311 WISTERIA DR,NaN,EAST PALO ALTO,CA,CALIFORNIA,94303,720-238-2873,DELAWARE,0,13778049,10097917,3680132,NaN,True,Other Technology
5951,1846636,"Neptune Topco Holdings, LLC","C/O NOVARIA HOLDINGS, LLC",6625 IRON HORSE BLVD,NORTH RICHLAND HILLS,TX,TEXAS,76180,817-381-3810,DELAWARE,0,5321392,5321392,0,NaN,True,Other
8517,1937975,"Taylor Meadows 712, LP","C/O TAYLOR MEADOWS 712 GP, LLC","1452 HUGHES ROAD, SUITE 200",GRAPEVINE,TX,TEXAS,76051,949-296-7502,TEXAS,100000,42000000,42000000,0,NaN,True,Other
5826,1787068,NDB Inc.,50 CALIFORNIA STREET,SUITE 1500,SAN FRANCISCO,CA,CALIFORNIA,94111,6502520002,CALIFORNIA,2523,1316715,1316715,0,NaN,True,Other Technology



Displaying sample of the final filtered data:


,Central Index Key,Name of the Entity,Address,Address - Specification,City,State or Country - Physical Location,State or Country - Full - Physical Location,Zip Code / Postal Code,Issuer Phone Number,State or Country - Legal Jurisdiction,...,Associated Address - Person 63,Associated Address - Specification - Person 63,Associated City - Person 63,Associated State or Country - Person 63,Associated State or Country - Full - Person 63,Associated Zip Code - Person 63,Relationship 1 with the Entity - Person 63,Clarification of the relationship - Person 63,Relationship 2 with the Entity - Person 18,Relationship 2 with the Entity - Person 19
7718,1946327,"SV Literati Investors, LLC",1023 Springdale Rd STE 1J,NaN,Austin,TX,TEXAS,78721,808-227-0349,DELAWARE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5316,1938046,"MANGOCEUTICALS, INC.",4131 N. CENTRAL EXPRESSWAY,SUITE 900,DALLAS,TX,TEXAS,75204,(833) 626-4679,TEXAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3156,1774765,"Exos Aerospace Systems & Technologies, Inc.",126 W. MALL DR.,NaN,CORSICANA,TX,TEXAS,75110,877-777-0703,TEXAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5444,1938946,MOCKINGBIRD TERRACE INVESTCO LLC,6221 E. MOCKINGBIRD LN.,NaN,DALLAS,TX,TEXAS,75214,(214) 724-8247,TEXAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6678,1941610,"Payverse, Inc.",5900 SEPULVEDA BLVD,SUITE 360,SHERMAN OAKS,CA,CALIFORNIA,91411,888-976-9099,DELAWARE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2112,1935143,Christina Real Estate Investors 5 Wealth Build...,"22653 PACIFIC COAST HIGHWAY, SUITE 1-8,",NaN,MALIBU,CA,CALIFORNIA,90265,(310) 456-7031,DELAWARE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6423,1938770,PC RISE ENCORE LLC,5424 SUNOL BLVD,NO. 10-1113,PLEASANTON,CA,CALIFORNIA,94566,(925) 336-0521,DELAWARE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2563,1937935,Del Lockhart Ventures LLC,302 CLARKS WAY,NaN,HUTTO,TX,TEXAS,78634,630-329-3944,TEXAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6089,1787352,"Nuclein, LLC","7301 N. FM 620 RD., SUITE 155-113",NaN,AUSTIN,TX,TEXAS,78726,512-879-3301,TEXAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8570,1937592,"Theom, Inc.",355 BRYANT STREET,SUITE 403,SAN FRANCISCO,CA,CALIFORNIA,94107,408-805-0009,DELAWARE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
